In [ ]:
#!pip install featuretools

In [ ]:
#!pip install numpy --upgrade

In [ ]:
import pandas as pd
import featuretools as ft

In [ ]:
df_seoul = pd.read_csv('https://raw.githubusercontent.com/Jiian/seoulbike/main/data/seoul_bike_agg_median.csv?token=AVLOXCNSXSMUIFTVUNKDIILBHGMUI')
#df_seoul = pd.read_csv('https://raw.githubusercontent.com/Jiian/seoulbike/main/data/seoul_bike_agg_mean.csv?token=AVLOXCOE7JOIREWTTZYFGV3BHGMRC')
df_seoul.head()

,Unnamed: 0,rent_count,year,month,day,day_of_week,season,temperature,humudity,holiday,wind_speed,visibility,dewpoint_temp,solar_radiation,rainfall,snowfall
0,1,9539,2017,12,1,4,Winter,-2.454167,37.5,No Holiday,1.537500,18707.500000,-13.545833,0.248750,0.0,0
1,2,8523,2017,12,2,5,Winter,1.325000,55.5,No Holiday,1.712500,14710.833333,-5.716667,0.263750,0.0,0
2,3,7222,2017,12,3,6,Winter,4.875000,84.5,No Holiday,1.612500,4557.500000,1.883333,0.125417,4.0,0
3,4,8729,2017,12,4,0,Winter,-0.304167,43.5,No Holiday,3.450000,13628.333333,-9.925000,0.282917,0.1,0
4,5,8307,2017,12,5,1,Winter,-4.458333,34.5,No Holiday,1.108333,19594.583333,-17.425000,0.035833,0.0,0


In [ ]:
df_seoul.dtypes

### Automated Feature Engineering

In [ ]:
df_x = df_seoul.drop(['rent_count','month','day','year'], axis=1)

In [ ]:
es = ft.EntitySet(id = 'bikes')

# adding a dataframe 
es.entity_from_dataframe(entity_id = 'seoul_bike', dataframe = df_seoul, index = 'Unnamed: 0')

Entityset: bikes
  Entities:
    seoul_bike [Rows: 353, Columns: 16]
  Relationships:
    No relationships

In [ ]:

es.normalize_entity(base_entity_id="seoul_bike",
                    new_entity_id="Season",
                    index="season")

es.normalize_entity(base_entity_id="seoul_bike",
                    new_entity_id="Holiday",
                    index="holiday")
es.normalize_entity(base_entity_id="seoul_bike",
                    new_entity_id="Day_of_week",
                    index="day_of_week")

print(es)

In [ ]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
target_entity = 'seoul_bike', 
max_depth = 2, 
verbose = 1, 
n_jobs = 3)

Built 246 features
Elapsed: 00:00 | Progress:   0%|          

/usr/local/lib/python3.7/dist-packages/featuretools/computational_backends/utils.py:154: UserWarning: 3 workers requested, but only 2 workers created.
  warnings.warn(warning_string)


EntitySet scattered to 2 workers in 3 seconds
Elapsed: 00:02 | Progress: 100%|██████████


In [ ]:
feature_matrix.columns

In [ ]:
from google.colab import files
feature_matrix.to_csv("seoul_bike_agg_median_FG.csv")
files.download("seoul_bike_agg_median_FG.csv")
#feature_matrix.to_csv("seoul_bike_agg_mean_FG.csv")
#files.download("seoul_bike_agg_mean_FG.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>